In [1]:
import pandas as pd
import time
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from glob import glob

In [2]:
# 設定Chrome Driver的執行檔路徑
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
options.add_experimental_option("prefs", {"profile.password_manager_enabled": False, "credentials_enable_service": False})
options.chrome_executable_path="C:\\Users\\user\\Desktop\\code\\python\\project\\chromedriver.exe"

In [3]:
# 建立 Driver物件實體，讓程式操作瀏覽器運作
driver=webdriver.Chrome(options=options)
# 要爬蟲的網頁
driver.get("https://www.imdb.com/search/title/?groups=top_100")
# driver.get("https://www.imdb.com/search/title/?title_type=feature&year=2023-01-01,2023-12-31")
# 設定程式碼的等待時間，以防網頁還沒有加載完就執行程式
driver.implicitly_wait(10)


In [4]:
# 爬取網頁中各個電影的資訊，名稱，連結，出產年分
title = []
link = []
year = []

# 在網頁原始碼html中，找尋class = lister-item-header的標籤
block = driver.find_elements(By.CLASS_NAME, "lister-item-header")
# 抓取50部電影的資訊
for i in range(0,50):
    ft = block[i].text
    # 找尋class = lister-item-index的標籤, 這邊是電影的索引
    fo = block[i].find_element(By.CLASS_NAME, 'lister-item-index').text
    # 找尋class = lister-item-year的標籤, 這邊是電影的年分
    fy = block[i].find_element(By.CLASS_NAME, 'lister-item-year').text
    # 取得電影的名稱
    ft = ft.replace(fo + " " , "")
    ft = ft.replace(fy , "")
    ft = ft.rstrip()
    # 把50部電影名稱存入title的list當中
    title.append(ft)
# 因為在IMDB中，有些電影的html格式不太一樣，所以需要用其他的方式取得電影的資訊，但是方法與上述一樣
blocks = driver.find_elements(By.CLASS_NAME, "lister-item")
for i in range(0,50): 
    ftitle = blocks[i].find_element(By.CLASS_NAME, 'lister-item-header').text
    forder = blocks[i].find_element(By.CLASS_NAME, 'lister-item-index').text
    fyear = ftitle[-6:]
    # 把50部電影的年份存入title的list當中
    year.append(fyear)
    flink = blocks[i].find_element(By.LINK_TEXT, title[i]).get_attribute('href')
    # 把50部電影的連結存入title的list當中
    link.append(flink)
# 檢查結果
print(title)
print(year)
print(link)


['蜘蛛人：穿越新宇宙', '蜘蛛人：新宇宙', '教父', '刺激1995', '法櫃奇兵', '星際效應', '四海好傢伙', '全面啟動', '黑暗騎士', '黑色追緝令', '復仇者聯盟：終局之戰', '寄生上流', '魔戒首部曲：魔戒現身', '綠色奇蹟', '鬥陣俱樂部', '神鬼無間', '搶救雷恩大兵', '進擊的鼓手', '惡棍特工', '阿甘正傳', '回到未來', '心靈捕手', '火線追緝令', '現代啟示錄', '駭客任務', '辛德勒的名單', '沉默的羔羊', '異形', '教父2', '終極追殺令', '決殺令', '噩夢輓歌', '小丑', '神鬼戰士', '星際大戰', '頂尖對決', '復仇者聯盟：無限之戰', '北非諜影', '美國心玫瑰情', '原罪犯', '鬼店', '北西北', '魔戒三部曲：王者再臨', '2001：太空漫遊', '十二怒漢', '獅子王', '記憶拼圖', '魔鬼終結者2', '黑暗騎士：黎明昇起', '飛越杜鵑窩']
['(2023)', '(2018)', '(1972)', '(1994)', '(1981)', '(2014)', '(1990)', '(2010)', '(2008)', '(1994)', '(2019)', '(2019)', '(2001)', '(1999)', '(1999)', '(2006)', '(1998)', '(2014)', '(2009)', '(1994)', '(1985)', '(1997)', '(1995)', '(1979)', '(1999)', '(1993)', '(1991)', '(1979)', '(1974)', '(1994)', '(2012)', '(2000)', '(2019)', '(2000)', '(1977)', '(2006)', '(2018)', '(1942)', '(1999)', '(2003)', '(1980)', '(1959)', '(2003)', '(1968)', '(1957)', '(1994)', '(2000)', '(1991)', '(2012)', '(1975)']
['https://www.imdb.com/title/tt9362722/?ref_=adv_l

In [5]:
# 這邊是要取得每部電影用戶評論的連結
user_review_links = []

# 進入每部電影個別的連結
for url in link:
        url = url
        # 這邊使用request模組來取得網頁的連線
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response = requests.get(url, headers = user_agent)
        # 這邊使用BeautifulSoup的模組來獲取網頁原始碼
        soup = BeautifulSoup(response.text, 'html.parser')
        # 在網頁原始碼html中，找尋標籤為a然後text為"User reviews"的程式碼，並獲取標籤a中的href超連結
        review_link = 'https://www.imdb.com'+soup.find('a', text = "User reviews").get('href')
        # 將每部電影的用戶評論連結存入user_review_links的list當中
        user_review_links.append(review_link)

# 製作一個csv檔確認每部電影的資訊，在壓縮檔中叫final.csv
top_50_data = {'Movie_name' : title, 'Year' : year, 'link' : link, 'user_review' : user_review_links,}
top50 = pd.DataFrame(data = top_50_data)
# encoding="utf_8_sig"是讓程式能夠讀出中文字
top50.to_csv('final.csv',encoding="utf_8_sig")
driver.quit()

C:\Users\user\AppData\Local\Temp\ipykernel_38460\4065235512.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  review_link = 'https://www.imdb.com'+soup.find('a', text = "User reviews").get('href')


In [ ]:
# 這邊就是最重要的部分，在有每部電影的用戶評論連結後，我們要進入這個連結，然後爬取最多200筆用戶分享的評論
for i in range(len(top50['user_review'])):
    # 這邊也是使用driver物件讓程式操作瀏覽器
    driver=webdriver.Chrome(options=options)
    driver.get(top50['user_review'][i])
    page = 1
    # IMDB中每個頁面只有25則評論，因此我們必須翻10頁來取得200筆以上的資訊
    while page < 10:
        try:
            # 用css_selector找尋'load-more-trigger'的位置
            css_selector = 'load-more-trigger'
            # 使用Driver物件自動點擊
            driver.find_element(By.ID, css_selector).click()
            time.sleep(3)
            page += 1
        except:
            pass
    # 尋找class = review-container的標籤
    review = driver.find_elements(By.CLASS_NAME, 'review-container')
    # 儲存星星數與評價的list
    rating = []
    lis = []
    cnt = 0
    # 設定最多找200筆資訊
    for n in range(0,250):
        try:
            if cnt >=200:
                break
            # 用戶評論必須同時具備rating和title的資料，否則略過並尋找下一筆
            frating = review[n].find_element(By.CLASS_NAME, 'rating-other-user-rating').text
            flist = review[n].find_element(By.CLASS_NAME, 'title').text

            rating.append(frating)
            lis.append(flist)
            cnt += 1
        except:
            continue
    # 將rating的資料從string轉成int
    for j in range(len(rating)):
        rating[j] = rating[j].replace('/10', "")
        rating[j] = int(rating[j])


    # 這邊是將每部電影的200則評論存入csv檔，用來確認有取得資訊，在壓縮檔中是叫folder_name的資料夾
    data = {'Rate' : rating, 'Review': lis}
    review = pd.DataFrame(data = data)
    movie = top50['Movie_name'][i]
    review['Movie_name'] = movie
    review.to_csv(f'C:\\Users\\user\\Desktop\\code\\python\\project\\folder_name\\final\\{i}.csv', encoding="utf_8_sig")
    driver.quit()